In [2]:
import pandas as pd
import pyarrow

In [3]:
data_0 = pd.read_parquet('/Users/elireadnour/Documents/Computer/Classes/machineLearning/MLFinal/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=0/part-0.parquet')
data_1 = pd.read_parquet('/Users/elireadnour/Documents/Computer/Classes/machineLearning/MLFinal/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=1/part-0.parquet')
data_2 = pd.read_parquet('/Users/elireadnour/Documents/Computer/Classes/machineLearning/MLFinal/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=2/part-0.parquet')
data_3 = pd.read_parquet('/Users/elireadnour/Documents/Computer/Classes/machineLearning/MLFinal/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=3/part-0.parquet')

In [9]:
datas = [data_0, data_1,data_2,data_3]
combined_df = pd.concat([data_0, data_1]) 

In [10]:
data_1.head()

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,170,0,0,2.112212,NaN,NaN,NaN,NaN,NaN,1.060330,...,-0.421823,-0.293646,-0.061842,-0.305413,-0.419151,-0.111796,-0.535104,-0.044332,-0.039061,-0.744789
1,170,0,1,2.760715,NaN,NaN,NaN,NaN,NaN,0.482468,...,3.111076,-0.075267,-0.359360,-1.270054,-0.018332,-0.040286,-1.417509,0.085840,0.487232,-0.124533
2,170,0,2,1.813596,NaN,NaN,NaN,NaN,NaN,1.020798,...,0.458474,-5.000000,-5.000000,0.194658,-5.000000,-5.000000,-5.000000,1.583400,0.018712,-1.055035
3,170,0,3,0.926893,NaN,NaN,NaN,NaN,NaN,0.510098,...,17.805511,3.336086,2.051951,2.400644,0.962730,-0.939277,1.845870,-2.372452,-1.663179,-4.585349
4,170,0,7,1.665231,NaN,NaN,NaN,NaN,NaN,0.547458,...,-0.249322,-0.707027,-0.344866,-1.248052,-0.129645,-3.145927,-0.452708,0.300044,0.489202,0.242737


In [ ]:
combined_df.head()
print("hello")

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,0,0,1,3.889038,NaN,NaN,NaN,NaN,NaN,0.851033,...,-0.281498,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504
1,0,0,7,1.370613,NaN,NaN,NaN,NaN,NaN,0.676961,...,-0.302441,2.965889,1.190077,-0.523998,3.849921,2.626981,5.000000,0.703665,0.216683,0.778639
2,0,0,9,2.285698,NaN,NaN,NaN,NaN,NaN,1.056285,...,-0.096792,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828
3,0,0,10,0.690606,NaN,NaN,NaN,NaN,NaN,1.139366,...,-0.296244,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516
4,0,0,14,0.440570,NaN,NaN,NaN,NaN,NaN,0.955200,...,3.418133,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.000000,-3.572820,-1.089123,-5.000000


In [6]:
data.dropna(axis=1)

,date_id,time_id,symbol_id,weight,feature_05,feature_06,feature_07,feature_09,feature_10,feature_11,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,0,0,1,3.889038,0.851033,0.242971,0.263400,11,7,76,...,-0.281498,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504
1,0,0,7,1.370613,0.676961,0.151984,0.192465,11,7,76,...,-0.302441,2.965889,1.190077,-0.523998,3.849921,2.626981,5.000000,0.703665,0.216683,0.778639
2,0,0,9,2.285698,1.056285,0.187227,0.249901,11,7,76,...,-0.096792,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828
3,0,0,10,0.690606,1.139366,0.273328,0.306549,42,5,150,...,-0.296244,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516
4,0,0,14,0.440570,0.955200,0.262404,0.344457,44,3,16,...,3.418133,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.000000,-3.572820,-1.089123,-5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1944205,169,848,19,3.438631,-0.028087,0.287438,0.118074,4,3,11,...,-0.166964,0.983339,-0.669860,0.272615,-3.676842,-1.221126,1.070584,0.465345,0.207483,0.874975
1944206,169,848,30,0.768528,-0.022584,0.442352,0.140746,81,2,534,...,-0.352810,0.992615,0.961595,1.089402,0.796034,0.488380,1.846634,-0.088542,-0.008324,-0.153451
1944207,169,848,33,1.354696,-0.024804,0.420692,0.136259,11,7,76,...,-0.239716,1.701618,0.757672,-5.000000,-3.174266,-1.110790,-3.349107,-0.407801,-0.185842,-0.931004
1944208,169,848,34,1.021797,-0.016138,0.303561,0.149970,42,5,150,...,-0.442859,-2.036891,-0.064228,1.919665,1.827681,0.872019,3.248694,0.254584,0.090288,0.434726


In [10]:
X = data.drop(['responder_6'], axis =1)
y = data['responder_6']

Data Preprocessing
- Heat maps Variance
- XGBoost --> Finds most highly correlated data

Random Forest